In [ ]:
using Pkg 
Pkg.add("GeometryBasics")
using GLMakie
using LinearAlgebra

# Function to calculate circumcircle in 2D
function circumcircle_2d(A, B, C)
    S_x = (1 / 2) * det([norm(A)^2 norm(B)^2 norm(C)^2 ; A[2] B[2] C[2] ; 1 1 1])
    S_y = (1 / 2) * det([A[1] B[1] C[1] ; norm(A)^2 norm(B)^2 norm(C)^2 ; 1 1 1])
    a = det([A[1] A[2] 1 ; B[1] B[2] 1 ; C[1] C[2] 1])
    b = det([A[1] A[2] norm(A)^2 ; B[1] B[2] norm(B)^2 ; C[1] C[2] norm(C)^2])

    circumcenter = (S_x / a, S_y / a)
    circumradius = sqrt((b / a) + norm([S_x, S_y])^2 / a^2)

    return circumcenter, circumradius
end

# Define the points of the 3D polygon
points_3d = [[12.726414, 13.161091, 1.224224], [12.666434, 12.797903, 0.384936], [12.565, 14.635716, -1.034951]]

# Single vertex in 3D
point_3d = [[12.565362, 14.635716, -1.029877]]

# Define the points of the 2D polygon
points_2d = [[-2.128461043458973, 13.161091], [-1.2870325276485568, 12.797903], [0.1364729854478327, 14.635716]]

# Single point in 2D
point_2d = [[0.13138608860044054, 14.635716]]

# Calculate circumcircle for 2D polygon
center_2d, radius_2d = circumcircle_2d(points_2d[1], points_2d[2], points_2d[3])

# Create a Figure
fig = Figure()

# convert from vector to vertices 
points_3d = [Point3f0(p...) for p in points_3d]
point_3d = [Point3f0(p...) for p in point_3d]
points_2d = [Point2f0(p...) for p in points_2d]
point_2d = [Point2f0(p...) for p in point_2d]
center_2d = Point2f0(center_2d)
points_2d = [Point2f0(p...) for p in points_2d]

# Add a 3D axis to the figure for the 3D polygon
ax3d = Axis3(fig[1, 1])
# Plot the 3D polygon using the mesh function
mesh!(ax3d, points_3d)
# Plot the 3D point on the 3D axis
scatter!(ax3d, point_3d)

# Add a 2D axis to the figure for the 2D polygon
ax2d = Axis(fig[1, 2])
# Plot the 2D polygon using the poly function
poly!(ax2d, points_2d)
# Plot the 2D point on the 2D axis
scatter!(ax2d, point_2d)

# Plot the circumcircle in 2D
circumcircle_2d_points = [(center_2d[1] + radius_2d * cos(t), center_2d[2] + radius_2d * sin(t)) for t in 0:0.01:2pi]
lines!(ax2d, circumcircle_2d_points)

# Display the figure
fig

In [ ]:
using Pkg
Pkg.add("GeometryBasics")
using GLMakie
using LinearAlgebra
using Statistics
using GeometryBasics

struct Triangle 
    A::Vector{Float64}
    B::Vector{Float64}
    C::Vector{Float64}
end

points_3d = [[12.726414, 13.161091, 1.224224], [12.666434, 12.797903, 0.384936], [12.565, 14.635716, -1.034951], [12.565362, 14.635716, -1.029877]]

In [ ]:
using Base
using LinearAlgebra
using GeometryBasics
using Distributions

include("util.jl")
include("interval.jl")
include("ray.jl")
include("material.jl")
include("hittable.jl")
include("aabb.jl")
include("sphere.jl")
include("triangle.jl")
include("hittable_list.jl")
include("bvh.jl") 
include("camera.jl")
include("scene.jl")

include("obj_reader.jl")

In [ ]:
obj_file = "../scenes/cottage_obj.obj"
sc = reader(obj_file)

world_l = hittable_list()

for face in sc.f_array
    if length(face.vertices) == 3
        t1 = Triangle(face.vertices[1], face.vertices[2], face.vertices[3], lambertian(color([0.0, 0.0, 0.8])))
        push!(world_l, t1)
    elseif length(face.vertices) == 4
        triangles = split_quad(face.vertices)
        for t in triangles
            push!(world_l, t)
        end
    else
        println("Unexpected face vertex count: ", length(face.vertices))
    end
end

println("World length: ", length(world_l.objects))

In [ ]:
triangle_to_point = [[Point3f0(t.A...), Point3f0(t.B...), Point3f0(t.C...)] for t in world_l.objects]

# render triangles using meshes 
using GLMakie
function render_triangles(triangles)
    fig = Figure()
    ax = Axis3(fig[1, 1])
    for t in triangles
        mesh!(ax, t)
    end
    fig
end

render_triangles(triangle_to_point)

In [44]:
node = bvh_node()
bvh_tree = bvh_node(world_l, node)

node_list = []

function print_bvh(world, indent = "")
	# if type is a bvh node 
	if typeof(world) == bvh_node
		println(indent, "BVH Node:")
		println(indent, "├─ Left:")
		print_bvh(world.left, indent * "│  ")
		println(indent, "└─ Right:")
		print_bvh(world.right, indent * "   ")
	else
        push!(node_list, world)
		println(indent, world)
	end
end

print_bvh(bvh_tree)

world_obj_list = world_l.objects

println("Node list length: ", length(node_list))
println("World object list length: ", length(world_obj_list))

BVH Node:
├─ Left:
│  BVH Node:
│  ├─ Left:
│  │  BVH Node:
│  │  ├─ Left:
│  │  │  BVH Node:
│  │  │  ├─ Left:
│  │  │  │  BVH Node:
│  │  │  │  ├─ Left:
│  │  │  │  │  BVH Node:
│  │  │  │  │  ├─ Left:
│  │  │  │  │  │  BVH Node:
│  │  │  │  │  │  ├─ Left:
│  │  │  │  │  │  │  BVH Node:
│  │  │  │  │  │  │  ├─ Left:
│  │  │  │  │  │  │  │  BVH Node:
│  │  │  │  │  │  │  │  ├─ Left:
│  │  │  │  │  │  │  │  │  Triangle(id = 7464532106920437)
│  │  │  │  │  │  │  │  └─ Right:
│  │  │  │  │  │  │  │     Triangle(id = 7464532106920437)
│  │  │  │  │  │  │  └─ Right:
│  │  │  │  │  │  │     BVH Node:
│  │  │  │  │  │  │     ├─ Left:
│  │  │  │  │  │  │     │  Triangle(id = 578050598296515)
│  │  │  │  │  │  │     └─ Right:
│  │  │  │  │  │  │        Triangle(id = 8643913268884988)
│  │  │  │  │  │  └─ Right:
│  │  │  │  │  │     BVH Node:
│  │  │  │  │  │     ├─ Left:
│  │  │  │  │  │     │  BVH Node:
│  │  │  │  │  │     │  ├─ Left:
│  │  │  │  │  │     │  │  Triangle(id = 51194059043069)

In [45]:
using Pkg
using Graphs
using GraphRecipes, Plots

function create_bvh_tree(world, tree)
    if typeof(world) == bvh_node
        tree["type"] = "bvh_node"
        tree["left"] = Dict()
        tree["right"] = Dict()
        create_bvh_tree(world.left, tree["left"])
        create_bvh_tree(world.right, tree["right"])
    else
        tree["type"] = "object"
        tree["object"] = "object"
    end
end

tree = Dict()

create_bvh_tree!(bvh_tree, tree)

println(tree)

Dict{Any, Any}("object" => "object", "type" => "object")


In [46]:
tri_to_id(tri_l) = map(x -> x.id, tri_l)

tl1 = tri_to_id(node_list)
tl2 = tri_to_id(world_obj_list)

println("Tri list 1: ", tl1)
println("Tri list 2: ", tl2)

# remove duplicates from both 
tl1 = unique(tl1)
tl2 = unique(tl2)

println("triangles to render ", length(tl1))
println("actual number of triangles ", length(tl2))

Tri list 1: [7464532106920437, 7464532106920437, 578050598296515, 8643913268884988, 51194059043069, 4903683140439157, 5486000360698364, 5027461204814361, 3584796021791005, 3584796021791005, 3571907175443558, 3992571309003972, 6692058139970282, 46156618647730, 2582540983124740, 3743617821521009, 6484861499493074, 6484861499493074, 3774099699303241, 7159286591135587, 8277460883597122, 1754381109580896, 6510574232598862, 3727259530193441, 2888894711114033, 9447059930796048, 4307037648198633, 6473751589697810, 9990951562773174, 9818174006290987, 4705934916027649, 85458592753619, 3940213866828851, 3940213866828851, 8697872355045048, 6366713467382475, 7300216970577189, 453116523600446, 8330598761095540, 3896238611703854, 209201958068110, 8650712072016508, 7951878062935371, 8179482595984806, 608658234805390, 2268495633524668, 9716426864404446, 2407441991687864, 9021989715820892, 9021989715820892, 3416106423267698, 7939558305807459, 4092818851724832, 6824596711882630, 4559342893473116, 6905126

In [47]:
common_ids = intersect(tl1, tl2)
setdiff(tl2, tl1)

for t in world_l.objects
    if t.id in setdiff(tl2, tl1)
        println("Triangle not in BVH: ", t.A, t.B, t.C)
    end
end

Triangle not in BVH: [12.726414, 13.161091, 1.224224][12.666434, 12.797903, 0.384936][12.565362, 14.635716, -1.029877]


In [48]:
triangle_to_point = [[Point3f0(t.A...), Point3f0(t.B...), Point3f0(t.C...)] for t in node_list]

render_triangles(triangle_to_point)